In [1]:
from pymongo import MongoClient
import json
from pymongo import MongoClient
from bson.json_util import dumps
import pandas as pd
import numpy as np

In [2]:
MONGO_HOST= 'mongodb://localhost/'  # assuming you have mongoDB installed locally
                                             # and a database called 'twitterdb'
client = MongoClient(MONGO_HOST)
db = client.twitterdb
data_lasso = db.twitter_search_lasso_after_elections
data_arauz = db.twitter_search_arauz_after_elections
#data_arauz_new = db.twitter_search_arauz_new 

In [3]:
def extract_data(client,database_search):
    columns_find = ["created_at","favorite_count","full_text","id_str","metadata.iso_language_code","retweeted_status.created_at","retweeted_status.full_text","user.created_at","user.favourites_count","user.followers_count","user.friends_count","user.id_str","user.location","user.verified","retweet_count"]
    data = database_search.find(no_cursor_timeout=True)
    data_tweets = None
    cont=0
    for tweet in data:
        dict_values_JSON = {}
        for column in columns_find:
            column_splited =  column.split(".")
            
            if len(column_splited)==2:
                if column_splited[0] in tweet:
                    data_key = tweet[column_splited[0]]
                    
                    if column_splited[1] in data_key:
                        dict_values_JSON[column] = [data_key[column_splited[1]]]
                    else:
                        dict_values_JSON[column] = [np.nan]
                else:
                    dict_values_JSON[column] = [np.nan]
            else:
                if column_splited[0] in tweet:
                    dict_values_JSON[column] = [tweet[column]]
                else:
                    dict_values_JSON[column] = [np.nan]

        if cont==0:
            data_tweets=pd.DataFrame(dict_values_JSON)
        else:
            df_auxiliar = pd.DataFrame(dict_values_JSON)
            data_tweets = pd.concat([data_tweets, df_auxiliar], axis=0)
            data_tweets = data_tweets.reset_index(drop=True)
        cont+=1
    return data_tweets

In [4]:
df_tweets_lasso = extract_data(client,data_lasso)
df_tweets_lasso.to_csv("tweets_lasso_after_elections.csv",index=False)

In [5]:
#df_tweets_lasso.to_csv("Twitter_search_Lasso.csv",index=False)

In [6]:
#df_tweets_lasso_new = extract_data(client,data_lasso_new)

In [7]:
#df_tweets_lasso_new.to_csv("Twitter_search_Lasso_new.csv",index=False)

In [8]:
df_tweets_arauz = extract_data(client,data_arauz)
df_tweets_arauz.to_csv("tweets_arauz_after_elections.csv",index=False)

In [9]:
#df_tweets_arauz.to_csv("Twitter_search_Arauz.csv",index=False)

In [10]:
#df_tweets_arauz_new = extract_data(client,data_arauz_new)

In [11]:
#df_tweets_arauz_new.to_csv("Twitter_search_Arauz_new.csv",index=False)

In [ ]:
#data_tweets = pd.concat([df_tweets_lasso, df_tweets_arauz], axis=0)
#data_tweets = data_tweets.reset_index(drop=True)
#data_tweets.to_csv("twitter_calidad.csv",index=False)